In [21]:
!pip install feature-engine -q

In [22]:
import pandas as pd

# to split the data sets:
from sklearn.model_selection import train_test_split

# to impute missing data with sklearn:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# to impute missing data with Feature-engine:
from feature_engine.imputation import MeanMedianImputer

In [23]:
# -rf 表示 "recursively" (遞迴) 和 "force" (強制)，會刪除整個資料夾及其所有內容
!rm -rf Feature-Engineering


In [24]:
# 刪除後，再次執行 clone 就不會報錯了
!git clone https://github.com/taipeihugo/Feature-Engineering.git -q

In [25]:
'''
# %cd 是 Colab/Jupyter 的 "魔術指令"，用來切換工作目錄 (Change Directory)
# 進入已經存在的資料夾
# %cd Feature-Engineering

# -q (quiet) 保持安靜模式，執行 "pull" 指令，從遠端 (origin) 的主分支 (main/master) 下載更新
!git pull -q
'''

'\n# %cd 是 Colab/Jupyter 的 "魔術指令"，用來切換工作目錄 (Change Directory)\n# 進入已經存在的資料夾\n# %cd Feature-Engineering\n\n# -q (quiet) 保持安靜模式，執行 "pull" 指令，從遠端 (origin) 的主分支 (main/master) 下載更新\n!git pull -q\n'

In [26]:
data = pd.read_csv("Feature-Engineering/credit_approval_uci.csv")
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,NaN,u,g,q,h,NaN,NaN,NaN,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("target", axis=1),
    data["target"],
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((483, 15), (207, 15))

In [28]:
X_train.isnull().sum()

,0
A1,4
A2,11
A3,68
A4,4
A5,4
A6,4
A7,4
A8,68
A9,68
A10,68


In [29]:
# Let's inspect the proportion of missing
# values per variable:

X_train.isnull().mean()

,0
A1,0.008282
A2,0.022774
A3,0.140787
A4,0.008282
A5,0.008282
A6,0.008282
A7,0.008282
A8,0.140787
A9,0.140787
A10,0.140787


In [30]:
# .select_dtypes() 是 pandas DataFrame 的一個方法，用於篩選特定資料型態的欄位
#
# 參數 exclude="O" 的意思是「排除 (exclude) 資料型態為 'O' 的欄位」
# 在 pandas 中，'O' (Object) 通常用來儲存字串 (string)
#
# 因此，X_train.select_dtypes(exclude="O") 會回傳一個新的 DataFrame，
# 裡面「只包含」非字串的欄位（例如：整數 int, 浮點數 float, 布林值 bool 等）
#
# .columns 會取得這個新 DataFrame 的所有「欄位名稱」
# .to_list() 則將這些欄位名稱轉換成一個 Python 列表
numeric_vars = X_train.select_dtypes(exclude="O").columns.to_list()
numeric_vars

['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

In [31]:
# Learn the variables median values:
# X_train[numeric_vars]
# (接續前一個步驟) 這裡會從 X_train 中，只選取 numeric_vars 列表裡包含的那些「數值型」欄位
#
# .median()
# 計算這些數值型欄位中「每一個欄位」的中位數
# （中位數是將所有數值排序後，位於最中間的那個值）
# 這會回傳一個 pandas Series，Index 是欄位名稱，Value 是該欄位的中位數
#
# .to_dict()
# 將這個 Series 轉換成一個 Python 字典 (dictionary)
# 字典的 "key" 是欄位名稱，"value" 是對應的中位數
#
# median_values = ...
# 將這個 {欄位: 中位數} 的字典，存到 median_values 這個變數中
median_values = X_train[numeric_vars].median().to_dict()

median_values

{'A2': 28.835, 'A3': 2.75, 'A8': 1.0, 'A11': 0.0, 'A14': 160.0, 'A15': 6.0}

In [32]:
# Replace missing data by the median:

X_train = X_train.fillna(value=median_values)
X_test = X_test.fillna(value=median_values)

In [33]:
# Corroborate absence of missing values:
X_train[numeric_vars].isnull().sum()

,0
A2,0
A3,0
A8,0
A11,0
A14,0
A15,0


In [34]:
# Corroborate absence of missing values:
X_test[numeric_vars].isnull().sum()

,0
A2,0
A3,0
A8,0
A11,0
A14,0
A15,0


# to impute missing data with sklearn

In [35]:
# to impute missing data with sklearn:

# 從 scikit-learn (sklearn) 的 impute (填補) 模組中，匯入 SimpleImputer 類別
# SimpleImputer 是一個用來填補資料集中缺失值 (NaN) 的工具
from sklearn.impute import SimpleImputer

# 從 scikit-learn (sklearn) 的 compose (組合) 模組中，匯入 ColumnTransformer 類別
# ColumnTransformer 允許你將不同的預處理步驟(例如填補、編碼)
# 應用到資料集的不同欄位上
from sklearn.compose import ColumnTransformer

In [57]:
# 使用Scikit-learn
# Split data into train and test set:

X_train, X_test, y_train, y_test = train_test_split(
    data.drop("target", axis=1),
    data["target"],
    test_size=0.3,
    random_state=0,
)

In [37]:
# numeric_vars = X_train.select_dtypes(exclude="O").columns.to_list()
numeric_vars

['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

In [38]:
# Make a list with the non-numerical variables:

remaining_vars = [var for var in X_train.columns if var not in numeric_vars]

remaining_vars

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [52]:
# Set up the imputer to replace missing data
# with the median:
# 設定 imputer (填補器) 來將缺失資料
# 替換為中位數:

# 建立一個 SimpleImputer 的實例 (instance)
# strategy="median" 表示我們選擇使用「中位數」作為填補缺失值 (NaN) 的策略
imputer = SimpleImputer(strategy="median")

# Indicate which variables to impute:
# 指示要填補哪些變數:

# 建立一個 ColumnTransformer (欄位轉換器) 的實例
# ColumnTransformer 的目的是讓我們可以「只對指定的欄位」套用特定的轉換
ct = ColumnTransformer(
    [
        # 參數 1: transformers 列表
        # 這裡我們定義了一個轉換步驟：
        # ("imputer",     <-- 這個轉換步驟的名稱 (可自訂)
        #  imputer,       <-- 要使用的轉換器物件 (即上面建立的 SimpleImputer)
        #  numeric_vars)  <-- 要套用這個 imputer 的欄位列表 (假設 numeric_vars 是一個包含所有數值欄位名稱的 list)
        ("imputer", imputer, numeric_vars)
    ],
    # 參數 2: remainder (剩餘欄位的處理方式)
    # "passthrough" (通過) 表示：
    # 任何不在 numeric_vars 列表中的欄位 (例如：類別型欄位)
    # 應該保持原樣 (pass through)，不要被丟棄 (drop) 或修改
    remainder="passthrough"
)

# Find the median value per variable:
# 找出每個變數的中位數:

# 呼叫 ColumnTransformer 的 .fit() 方法，並傳入「訓練資料集」 X_train
# 這個步驟會執行：
# 1. ct (ColumnTransformer) 會根據 numeric_vars 抓出 X_train 中的數值欄位。
# 2. ct 會將這些數值欄位傳給 "imputer" (SimpleImputer)。
# 3. SimpleImputer 會計算「每一個」數值欄位的中位數，並將這些中位數「記住」(儲存在物件內部)。
#
# 注意：.fit() 只是學習資料 (計算中位數)，它「不會」修改 X_train 本身。
# 真正執行填補動作的是 .transform() 方法。
ct.fit(X_train); # 加;

In [55]:
# Check the medians that will be used in
# the imputation:
# 檢查將用於填補的中位數：
#
#
# ct 是我們已經 .fit() 過的 ColumnTransformer 物件
#
# .named_transformers_ (結尾有底線 _)
# 這是一個在 .fit() 之後才會出現的屬性，
# 它是一個字典，儲存了所有「有被命名的」轉換器 (transformer)
#
# .imputer
# 透過我們之前設定的名稱 "imputer" (在 ColumnTransformer 的 [("imputer", ...)] 中設定的)，
# 從 .named_transformers_ 字典中取出對應的 SimpleImputer 物件
#
# .statistics_ (結尾有底線 _)
# 這是 SimpleImputer 在 .fit() 之後才會有的屬性，
# 它儲存了計算出來的統計值。
#
# 總結：
# 因為我們的 imputer 策略是 "median" (中位數)，
# 所以這行程式碼會回傳一個陣列 (array)，
# 裡面包含了 .fit(X_train) 時，imputer 所計算並儲存下來的「每一個數值欄位的中位數」

ct.named_transformers_.imputer.statistics_

array([ 28.835,   2.75 ,   1.   ,   0.   , 160.   ,   6.   ])

In [58]:
# Replace missing data:
# 取代缺失資料:

# 使用 .fit() 訓練好的 ColumnTransformer (ct) 來「轉換」訓練集 X_train
# .transform() 方法會執行以下動作：
# 1. 找到 numeric_vars 中的所有缺失值 (NaN)。
# 2. 用先前 .fit() 時計算並儲存的「中位數」來填補這些缺失值。
# 3. 根據 remainder="passthrough" 設定，保留所有非數值欄位。
#
# 注意：ct.transform() 預設會回傳一個 NumPy 陣列 (array)，
# 這裡將 X_train 變數「覆蓋」為這個新的、已填補好缺失值的 NumPy 陣列。
X_train = ct.transform(X_train)

# 使用「同一個」ct 轉換器 (包含用 X_train 學到的中位數)
# 來轉換「測試集」 X_test
#
# 關鍵點：我們「必須」使用從 X_train 學到的中位數來填補 X_test，
# 這樣才能確保資料處理的一致性，並避免「資料洩漏」(data leakage)。
# X_test 同樣會被轉換成一個 NumPy 陣列。
X_test = ct.transform(X_test)

X_train

array([[46.08, 3.0, 2.375, ..., 't', 't', 'g'],
       [15.92, 2.875, 0.085, ..., 'f', 'f', 'g'],
       [36.33, 2.125, 0.085, ..., 't', 'f', 'g'],
       ...,
       [19.58, 0.665, 1.665, ..., 'f', 'f', 'g'],
       [22.83, 2.29, 2.29, ..., 't', 't', 'g'],
       [40.58, 3.29, 3.5, ..., 'f', 't', 's']], dtype=object)

In [42]:
# Convert returned array to a pandas dataframe:

# 呼叫 pandas (pd) 的 DataFrame 建構子 (constructor)
#
# Scikit-learn 的 .transform() 方法會回傳一個 NumPy 陣列 (array)，
# 這個陣列「不包含」欄位名稱。
#
# 為了後續分析方便 (例如查看特定欄位)，
# 我們使用 pd.DataFrame() 函式將這個陣列「包裝」回 DataFrame，
# 並手動將欄位名稱指定回去。
X_train = pd.DataFrame(
    # 參數 1 (data):
    # 傳入 X_train (它在上一動中已被 .transform() 轉換為 NumPy 陣列
    X_train,

    # 參數 2 (columns):
    # 傳入欄位名稱的列表。
    #
    # 這裡的「順序」至關重要：
    #
    # 1. ColumnTransformer 預設會將「有被轉換器處理」的欄位
    #    (也就是 numeric_vars) 放在輸出陣列的「最前面」。
    #
    # 2. 接著，它會將設定為 remainder="passthrough" (通過) 的欄位
    #    (也就是 remaining_vars) 放在「後面」。
    #
    # 3. 因此，我們必須使用 `numeric_vars + remaining_vars`
    #    這個順序來串接兩個列表，才能正確還原所有欄位的名稱。
    #    (這裡假設 remaining_vars 是一個您先前已定義好的列表，
    #     包含了所有非數值型態的欄位名稱)
    columns = numeric_vars + remaining_vars,
)

X_train.head()

,A2,A3,A8,A11,A14,A15,A1,A4,A5,A6,A7,A9,A10,A12,A13
0,46.08,3.0,2.375,8.0,396.0,4159.0,a,u,g,c,v,t,t,t,g
1,15.92,2.875,0.085,0.0,120.0,0.0,a,u,g,q,v,f,f,f,g
2,36.33,2.125,0.085,1.0,50.0,1187.0,b,y,p,w,v,t,t,f,g
3,22.17,0.585,0.0,0.0,100.0,0.0,b,y,p,ff,ff,f,f,f,g
4,57.83,7.04,14.0,6.0,360.0,1332.0,b,u,g,m,v,t,t,t,g


In [43]:
# Corroborate absence of missing values:

X_train[numeric_vars].isnull().sum()

,0
A2,0
A3,0
A8,0
A11,0
A14,0
A15,0


In [44]:
# Convert returned array to a pandas dataframe:

X_test = pd.DataFrame(
    X_test,
    columns=numeric_vars + remaining_vars,
)

X_test.head()

,A2,A3,A8,A11,A14,A15,A1,A4,A5,A6,A7,A9,A10,A12,A13
0,45.83,10.5,5.0,7.0,0.0,0.0,a,u,g,q,v,t,t,t,g
1,64.08,20.0,17.5,9.0,0.0,1000.0,b,u,g,x,h,t,t,t,g
2,31.25,3.75,0.625,9.0,181.0,0.0,a,u,g,cc,h,t,t,t,g
3,39.25,9.5,6.5,14.0,240.0,4607.0,b,u,g,m,v,t,t,f,g
4,26.17,2.0,0.0,0.0,276.0,1.0,a,u,g,j,j,f,f,t,g


In [45]:
# Corroborate absence of missing values:

X_test[numeric_vars].isnull().sum()

,0
A2,0
A3,0
A8,0
A11,0
A14,0
A15,0


# to impute missing data with Feature-engine



In [59]:
# 使用Feature-engine
# Split data into train and test set:

X_train, X_test, y_train, y_test = train_test_split(
    data.drop("target", axis=1),
    data["target"],
    test_size=0.3,
    random_state=0,
)

In [60]:
# Set up the imputer to replace missing data
# with the median:
# (設定 imputer 來將缺失資料替換為中位數)

# 建立一個 MeanMedianImputer 的實例 (instance)
# 這是 'feature-engine' 函式庫中用來填補缺失值的工具
imputer = MeanMedianImputer(
    # 參數 1: imputation_method="median"
    # 指定填補策略為 "median" (中位數)
    # (另一個常見選項是 "mean" (平均數))
    imputation_method="median",

    # 參數 2: variables=numeric_vars
    # 明確指定這個 imputer「只」應該應用在 numeric_vars 列表中的那些欄位
    # (這與 sklearn 的 ColumnTransformer 概念不同，
    #  feature-engine 的 transformer 通常會直接指定要處理的欄位)
    variables=numeric_vars,
)

# Find the median values:
# (找出中位數值)

# 呼叫 .fit() 方法，並傳入「訓練資料集」 X_train
# 這個步驟會執行：
# 1. imputer 會查看 X_train 中 numeric_vars 指定的那些欄位。
# 2. 計算「每一個」指定欄位的中位數。
# 3. 將這些計算出來的中位數 (欄位名稱: 中位數) 儲存在 imputer 物件內部的 .imputer_dict_ 屬性中，
#    以便後續 .transform() 步驟使用。
#
# 注意：.fit() 同樣只學習資料，並「不會」修改 X_train 本身。
imputer.fit(X_train);

In [48]:
# The median values per variable:
# (每個變數的中位數值)

# imputer 是我們剛剛 .fit() 過的 MeanMedianImputer 物件
#
# .imputer_dict_
# 這是 'feature-engine' 函式庫的一個慣例
#
# 結尾的底線 ( _ ) 表示這是 .fit() 之後才產生的「學習結果」屬性
#
# _dict_ 表示這個屬性是一個「字典 (dictionary)」
#
# 這行程式碼會印出一個字典，
# 內容是 { 欄位名稱: 該欄位的中位數 }

imputer.imputer_dict_

{'A2': 28.835, 'A3': 2.75, 'A8': 1.0, 'A11': 0.0, 'A14': 160.0, 'A15': 6.0}

In [49]:
# Replace missing data with the median:
# (用中位數取代缺失資料)

# 呼叫 imputer 的 .transform() 方法，並傳入 X_train
# 這個步驟會：
# 1. 找到 X_train 中，之前 .fit() 時指定要處理的欄位 (numeric_vars) 裡的缺失值 (NaN)
# 2. 使用 .fit() 時儲存下來的「訓練集的中位數」 (imputer.imputer_dict_) 來填補這些缺失值
#
# 'feature-engine' 的 transform 方法會回傳一個已填補好的 DataFrame
# 這裡將 X_train 變數「覆蓋」為填補後的結果
X_train = imputer.transform(X_train)

# 使用「同一個」imputer 物件 (包含從 X_train 學到的中位數)
# 來轉換「測試集」 X_test
#
# 關鍵：這是為了確保資料處理的一致性，並防止「資料洩漏」(data leakage)
# 我們必須用「訓練集」的統計數據 (中位數) 來填補「測試集」
X_test  = imputer.transform(X_test)

In [50]:
# Corroborate absence of missing values:
# 確認沒有缺失值

# X_train[numeric_vars]
# 從 X_train DataFrame 中，只選取我們剛剛處理過的「數值型」欄位
#
# .isnull()
# 逐一檢查這些欄位中的每一個值，
# 如果該值是缺失值 (NaN)，則標記為 True
# 如果該值不是缺失值，則標記為 False
# 這會回傳一個和 X_train[numeric_vars] 同樣大小、但充滿 True/False 的 DataFrame
#
# .sum()
# 將 .isnull() 產生的 True (視為 1) 和 False (視為 0) 進行「逐欄」加總
#
# 總結：
# 這行程式碼會計算「每一個數值欄位」中，
# 「總共有多少個缺失值 (NaN)」
#
# 如果填補成功，這裡印出的結果應該是每個欄位都是 0
X_train[numeric_vars].isnull().sum()

,0
A2,0
A3,0
A8,0
A11,0
A14,0
A15,0


In [51]:
# Corroborate absence of missing values:

X_test[numeric_vars].isnull().sum()

,0
A2,0
A3,0
A8,0
A11,0
A14,0
A15,0
